In [1]:
import numpy as np
import pandas as pd
import time
import os
import json
import matplotlib.pyplot as plt
import glob

from pitch_methods import inference, format_onset_offset, get_pyin, get_harmo, get_crepe, visualise_results, visualise_results_librosa
from peak_detection import find_raw_peak

In [2]:
"""
Load the metadata that includes a list of which files are valid
"""
with open('all_metadata.json', 'r') as f:
    all_metadata = json.load(f)

In [4]:
"""
This function is defined to
1) load data from the metadata file
2) process using the various pitch-estimation methods
3) prepare that data to plot
4) save the graph to a folder
"""

def plot_graph(metadata):
    audio_path = metadata['path']
    transcription = metadata['transcription']
    onset_times = []
    for word_index in metadata['alignment']:
        onset_times.append([
            metadata['alignment'][word_index]['text'], 
            metadata['alignment'][word_index]['onset'], 
            metadata['alignment'][word_index]['offset'],
            metadata['alignment'][word_index]['type'],
            metadata['alignment'][word_index]['mispronunciation']
            ])
    onset_times = pd.DataFrame(onset_times, columns=['word', 'onset', 'offset', 'type', 'misp'])

    # calculations
    df_raw_peaks = find_raw_peak(audio_path, onset_times, cutoff_freq = 500)
    df_pyin = get_pyin(audio_path, onset_times)
    df_crepe = get_crepe(audio_path, onset_times, shortest_slice = 20)
    df_harmo = get_harmo(audio_path, onset_times)

    # define plotting parameters
    raw_peak_dict = {'t': df_raw_peaks['t'],
                    'f': df_raw_peaks['f'],
                    'weight': df_raw_peaks['h'],
                    'colour': 'red',
                    'size': 5}
    pyin_dict = {'t': df_pyin['t'],
                'f': df_pyin['f'],
                'weight': df_pyin['p'],
                'colour': 'lime',
                'size': 32}
    crepe_dict = {'t': df_crepe['t'],
                'f': df_crepe['f0'],
                'weight': df_crepe['conf'],
                'colour': 'cyan',
                'size': 32}
    harmo_dict = {'t': df_harmo['t'],
                'f': df_harmo['f'],
                'weight': df_harmo['act'],
                'colour': 'fuchsia',
                'size': 32}
    methods_dict = {'raw_peaks': raw_peak_dict,
                    'pyin': pyin_dict,
                    'harmo': harmo_dict,
                    'crepe': crepe_dict}
    
    # plot
    filename = audio_path.split('/')[-1].split('.')[0]
    visualise_results_librosa(audio_path, methods_dict, onset_times, cutoff = 500, filename = filename)

In [9]:
# iterate over files to generate graphs
errors = []
folder_index = 0
for folder in all_metadata:
    file_index = 0
    for file in all_metadata[folder]:
        metadata = all_metadata[folder][file]
        try:
            plot_graph(metadata)
        except:
            errors.append(metadata['path'])
        print(file_index,'/',len(all_metadata[folder]), 'of', folder_index,'/',len(all_metadata), metadata['path'])
        file_index += 1
    folder_index += 1

0 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S001.wav
1 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S002.wav
2 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S003.wav
3 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S004.wav
4 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S005.wav
5 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S006.wav
6 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S007.wav
7 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S008.wav
8 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S009.wav
9 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S010.wav
10 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S011.wav
11 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S012.wav
12 / 61 of 0 / 65 ./Chinese_DPA/audio/001cb1/001cb1_AY2122S1_week3_S013.wav
13 / 61 of 0 / 65 ./Ch